In [10]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                     output_attentions=True,
                                                     output_hidden_states=True)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Torch not compiled with CUDA enabled

In [3]:
import pandas as pd

df = pd.read_csv('../datasets/combined/combined_dataset_200samples_50-50split.csv')
first_article = df.iloc[0]['article']

print(f"First line of the dataset: {first_article}")

First line of the dataset: Rot-Weiss Essen besiegt Zweitligist Düsseldorf und zieht als einziger Viertligist ins Achtelfinale ein. Wolfsburg, Stuttgart und Bremen gewinnen. Der frühere Bundesligist Rot-Weiss Essen steht nach der Überraschung im Nachbarschafts-Duell mit Fortuna Düsseldorf erstmals seit zwölf Jahren wieder im DFB-Pokal-Achtelfinale. Der bislang noch ungeschlagene Spitzenreiter der Regionalliga West kam am Mittwoch zu einem glücklichen 3:2 (2:1) in der zweiten Pokalrunde gegen den Absteiger aus der Fußball-Bundesliga. Simon Engelmann (15. Minute), Kapitän Marco Kehl-Gomez (39.) und der eingewechselte Oguzhan Kefkir (70.) nutzten die einzigen drei echten Chancen des Viertligisten eiskalt aus.
Die deutlich feldüberlegene Fortuna kam trotz ihres höheren Aufwands nur zum zwischenzeitlichen 1:1 und zum späten Elfmeter-Tor jeweils durch Torjäger Rouwen Hennings (36./87.). Damit scheiterte das Team von Trainer Uwe Rösler wie schon in der Vorsaison an einem Viertligisten. Im März

In [5]:
import tensorflow as tf


encoded_dict = tokenizer.encode_plus(
    first_article,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    return_attention_mask=True, # set 0 if token is not present (padding), set 1 if token is present
    return_tensors='tf',
    padding='max_length'  # Ensure all sequences are of the same length
)

encoded_dict['input_ids']
encoded_dict['attention_mask']



print(f"Original first article: {first_article}")
print(f"Tokenized first article: {encoded_dict['input_ids']}")

print(f"Length of original first article: {len(first_article)}")
print(f"Length of tokenized first article: {len(encoded_dict['input_ids'])}")

Original first article: Rot-Weiss Essen besiegt Zweitligist Düsseldorf und zieht als einziger Viertligist ins Achtelfinale ein. Wolfsburg, Stuttgart und Bremen gewinnen. Der frühere Bundesligist Rot-Weiss Essen steht nach der Überraschung im Nachbarschafts-Duell mit Fortuna Düsseldorf erstmals seit zwölf Jahren wieder im DFB-Pokal-Achtelfinale. Der bislang noch ungeschlagene Spitzenreiter der Regionalliga West kam am Mittwoch zu einem glücklichen 3:2 (2:1) in der zweiten Pokalrunde gegen den Absteiger aus der Fußball-Bundesliga. Simon Engelmann (15. Minute), Kapitän Marco Kehl-Gomez (39.) und der eingewechselte Oguzhan Kefkir (70.) nutzten die einzigen drei echten Chancen des Viertligisten eiskalt aus.
Die deutlich feldüberlegene Fortuna kam trotz ihres höheren Aufwands nur zum zwischenzeitlichen 1:1 und zum späten Elfmeter-Tor jeweils durch Torjäger Rouwen Hennings (36./87.). Damit scheiterte das Team von Trainer Uwe Rösler wie schon in der Vorsaison an einem Viertligisten. Im März ha

In [8]:
output = model(encoded_dict['input_ids'], attention_mask=encoded_dict['attention_mask'])

In [9]:
output.hidden_states 

(<tf.Tensor: shape=(1, 512, 768), dtype=float32, numpy=
 array([[[-0.48230153,  0.32629097,  0.37618014, ...,  0.28631747,
          -0.90260947,  0.01283996],
         [-1.9574678 ,  0.11998283, -0.52765363, ..., -1.0067387 ,
          -0.75383043, -0.2297786 ],
         [-1.3107111 ,  2.3562386 ,  1.7727066 , ...,  0.44063115,
           0.30648267, -0.75741875],
         ...,
         [ 1.9676633 , -0.23459473,  1.3176453 , ..., -0.19063394,
          -0.9960441 , -0.3939369 ],
         [-0.32624888,  0.31903368,  1.6460272 , ...,  0.36437565,
           0.18716516,  0.39599824],
         [-0.51195425,  0.46750206,  0.5430804 , ...,  0.03593849,
          -1.0480646 , -1.6619052 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 512, 768), dtype=float32, numpy=
 array([[[-0.28608802,  0.4927413 ,  0.20727777, ...,  0.4563554 ,
          -0.48655015,  0.10393047],
         [-1.5829247 ,  0.6943203 , -0.10037682, ..., -0.96523136,
          -0.77860594, -0.012145  ],
         [-1.0254295 , 